In [11]:
#Add all the imports here
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import time
import logging
import warnings

from datetime import datetime, timedelta, time
from itertools import combinations

In [12]:
# Silence yfinance logs
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

# Silence warnings
warnings.filterwarnings("ignore")

In [13]:
#To get data from finanace

def get_data(symbol, date,interval):
  ticker = yf.Ticker(symbol)
  start_date = datetime.strptime(date, "%Y-%m-%d")
  end_date = start_date + timedelta(days=1)
  try:
    data = ticker.history(start=start_date, end=end_date, interval=interval);
  except Exception as e:
    return None
  if data is None or data.empty:
    return None
  return data

def extract_ticker(df, ticker):
    for level in range(df.columns.nlevels):
        if ticker in df.columns.get_level_values(level):
            return df.xs(ticker, axis=1, level=level)
    raise KeyError(f"{ticker} not found in columns")

def fix_ist(df):
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC")
    return df.tz_convert("Asia/Kolkata")

def get_single_symbol_day(symbol, date, interval):
    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)

    df = yf.download(
        symbol,
        start=start,
        end=end,
        interval=interval,
        progress=False
    )

    if df.empty:
        return None

    df = extract_ticker(df, symbol)

    return fix_ist(df)

def get_multiple_symbols_day(symbols, date, interval, batch_size=50):
    start = datetime.strptime(date, "%Y-%m-%d")
    end = start + timedelta(days=1)

    def chunk(lst, size):
        for i in range(0, len(lst), size):
            yield lst[i:i + size]

    all_data = []

    for batch in chunk(symbols, batch_size):
        df = yf.download(
            batch,
            start=start,
            end=end,
            interval=interval,
            group_by="ticker",
            progress=False
        )

        if not df.empty:
            all_data.append(df)

        time.sleep(3)  # REQUIRED for intraday safety

    if not all_data:
        return None

    return fix_ist(pd.concat(all_data, axis=1))

In [14]:
#Graph helpers

def create_candlestick_figure(ticker_symbol,date):
    fig = go.Figure()

    fig.update_layout(
        title=f'{ticker_symbol} Candlestick Chart',
        xaxis_title=f"Date:{date}",
        yaxis_title="Price",
        dragmode="pan",                 # ✅ allow panning
        xaxis=dict(
            type="category",
            rangeslider=dict(visible=False),
            tickmode="array",
            showticklabels=False,  # ✅ hides x-axis labels
        )
    )

    # Enable zoom in both directions
    fig.update_xaxes(fixedrange=False)
    fig.update_yaxes(fixedrange=False)

    return fig


def update_candlestick_data(
    fig: go.Figure,
    historical_data: pd.DataFrame
):
    candle = go.Candlestick(
        x=historical_data.index,
        open=historical_data["Open"],
        high=historical_data["High"],
        low=historical_data["Low"],
        close=historical_data["Close"],
        increasing_line_color="green",
        decreasing_line_color="red",
    )
    fig.add_trace(candle)
    return fig

def add_hline(fig: go.Figure,value,annot,color="blue",annot_pos="top left"):
    fig.add_hline(
      y= value,
      line=dict(color=color, width=1, dash="dash"),
      annotation_text=f"{annot}",
      annotation_position=annot_pos
    )
    return fig

def add_vzone(fig: go.Figure,start_dt, end_dt):
    fig.add_vrect(
      x0=start_dt,
      x1=end_dt,
      fillcolor="rgba(255,0,0,0.12)",  # translucent red band
      line_width=0,
      layer="below"
    )
    return fig


def add_marker(fig: go.Figure,time,price,text):
    fig.add_annotation(
      x=time,
      y=price,
      text=text,
      showarrow=True,
      arrowhead=2,
      arrowsize=1.2,
      arrowcolor="crimson",
      ax=-40, ay=-40,
      bgcolor="rgba(255,255,255,0.8)",
      bordercolor="crimson",
      borderwidth=1,
      font=dict(size=12, color="black"),
      xref="x", yref="y"
    )
    return fig

def add_header(fig: go.Figure,text):
  fig.add_annotation(
      text=text,
      xref="paper", yref="paper",
      x=0.5, y=1.06,
      showarrow=False,
      font=dict(size=16)
  )
  return fig

In [15]:
# -------- GLOBAL COUNTERS --------
GLOBAL_STATE = {
    "win": 0,
    "loss": 0,
    "pnl" :  0,
    "tot_trade" : 0
}

In [16]:
def fig_data_support_resistance(fig: go.Figure,data,support,resistance):
  fig = update_candlestick_data(fig, historical_data=data)
  fig = add_hline(fig,support,"support")
  fig = add_hline(fig,resistance,"resistance")
  return fig

def fig_start_zone(fig: go.Figure,date):
  start_dt = pd.Timestamp(f"{date} 10:10:00").tz_localize("Asia/Kolkata")
  end_dt   = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")
  fig = add_vzone(fig, start_dt, end_dt)
  return fig

def fig_entry_target_stop(fig: go.Figure,position,entry,entry_index,stop,target):
  fig = add_marker(fig, entry_index, entry, f"Entry({position}):{entry}")
  fig = add_hline(fig, stop, f"Stoploss:{stop}", "red", "top right")
  fig = add_hline(fig, target, f"Target:{target}", "green", "top right")
  return fig

def update_trade_result(fig, result, entry, exit_price, index, position):
    global GLOBAL_STATE

    win = GLOBAL_STATE["win"]
    loss = GLOBAL_STATE["loss"]
    pnl = GLOBAL_STATE["pnl"]
    tot_trade = GLOBAL_STATE["tot_trade"]

    if result == "win":
        win += 1
        current_pct = round(
            ((entry - exit_price) / entry) * 100 if position == "Short"
            else ((exit_price - entry) / entry) * 100,
            2
        )
    else:
        loss += 1
        current_pct = round(
            ((entry - exit_price) / entry) * 100 if position == "Short"
            else ((exit_price - entry) / entry) * 100,
            2
        )

    pnl += current_pct
    tot_trade += 1

    GLOBAL_STATE.update({
        "win": win,
        "loss": loss,
        "pnl": pnl,
        "tot_trade": tot_trade
    })

    fig = add_marker(fig, index, exit_price, result.capitalize())
    fig = add_header(
        fig,
        f"Current: {result.capitalize()} | PnL:{current_pct}%  ||  "
        f"Total: Win:{win} Loss:{loss}  PnL:{pnl}%"
    )

    return fig


def evaluate_trade(fig, data, start_i, entry, stop, target, position):
  i = start_i

  while i < len(data):
      row = data.iloc[i]
      index = row.name
      high, low = row.High, row.Low

      # WIN
      if high > target and target > low:
        return update_trade_result(fig, "win", entry, target, index, position)

      # LOSS
      if high > stop and stop > low:
        return update_trade_result(fig, "loss", entry, stop, index, position)

      i += 1

  return fig

def strategy1(ticker_symbol, date, data_1h, data_5m):
  global GLOBAL_STATE
  win = GLOBAL_STATE["win"]
  loss = GLOBAL_STATE["loss"]
  pnl = GLOBAL_STATE["pnl"]
  tot_trade = GLOBAL_STATE["tot_trade"]

  fig = create_candlestick_figure(ticker_symbol,date)

  # --------- LEVELS (Higher TF) ---------
  support    = data_1h["High"].iloc[0]
  resistance = data_1h["Low"].iloc[0]

  # --------- LOWER TF DATA ---------
  data = data_5m

  fig = fig_start_zone(fig,date)
  fig = fig_data_support_resistance(fig, data, support, resistance)

  start_tf = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")

  # --------- STATE ---------
  short_state = {"breakout": False, "risk": None}
  long_state  = {"breakout": False, "risk": None}

  # --------- ITERATION ---------
  i = 0
  while i < len(data):
    row = data.iloc[i]
    index = row.name

    if index < start_tf:
      i += 1
      continue

    close, high, low = row.Close, row.High, row.Low

    # =========== SHORT SETUP ==================
    if not short_state["breakout"]:
      if close > support:
        short_state.update({"breakout": True, "risk": high})
        fig = add_marker(fig, index, close, "Breakout (Short)")

    else:
      short_state["risk"] = max(short_state["risk"], high)

      if close < support:
        entry = close
        stop = short_state["risk"]
        target = entry + (entry - stop) * 2

        fig = fig_entry_target_stop(fig, "Short", entry, index, stop, target)
        i += 1
        fig = evaluate_trade(fig, data, i, entry, stop, target, "Short")

        short_state = {"breakout": False, "risk": None}
        i += 1
        #continue

        break   #single trade per day

    # ================= LONG =================
    if not long_state["breakout"]:
      if close < resistance:
        long_state["breakout"] = True
        long_state.update({"breakout": True, "risk": low})
        fig = add_marker(fig, index, close, "Breakout (Long)")

    else:
      long_state["risk"] = min(long_state["risk"], low)

      if close > resistance:
        entry = close
        stop = long_state["risk"]
        target = entry - (stop - entry) * 2

        fig = fig_entry_target_stop(fig, "Long", entry, index, stop, target)
        i += 1
        fig = evaluate_trade(fig, data_5m, i + 1, entry, stop, target, "Long")

        long_state = {"breakout": False, "risk": None}
        i += 1
        #continue

        break   #single trade per day

    i += 1

  return fig


In [17]:
def get_ticker_list():
    # Authenticate (Colab)
    from google.colab import auth
    auth.authenticate_user()

    import gspread
    import pandas as pd
    from google.auth import default

    # Authorize
    creds, _ = default()
    gc = gspread.authorize(creds)


    # Open sheet
    sheet = gc.open_by_url(
        "https://docs.google.com/spreadsheets/d/1SAS2_nj_BN22vsYMbpG79MgbiBLI1YaVkRejYMWe3hE/edit"
    ).sheet1

    # Read data into DataFrame
    df = pd.DataFrame(sheet.get_all_records())

    # Get First column → clean → append ".NS"
    s = (
    df.iloc[:, 0]
    .dropna()
    .astype(str)
    .str.strip()
    )

    tickers = (
        s.where(s.str.endswith(".NS"), s + ".NS")
        .loc[s != ""]
        .tolist()
    )

    return tickers


In [18]:
tz="Asia/Kolkata"

def run_range(ticker_symbol, days_back, show_fig=False):
    today = pd.Timestamp.now(tz=tz).normalize()
    start = today - pd.Timedelta(days=days_back-1)
    dates = pd.date_range(start=start, end=today, freq="D", tz=tz)

    total = len(dates)
    print(f"Running {ticker_symbol} from {dates[0].date()} to {dates[-1].date()} ({total} days)…")

    auto = False
    for i, dt in enumerate(dates, start=1):
        date_str = dt.strftime("%Y-%m-%d")  # matches your strategy1 signature
        if show_fig:
          print(f"\n[{i}/{total}] strategy1({ticker_symbol!r}, {date_str})")

        try:
            fig = strategy1(ticker_symbol, date_str)  # <-- your existing function
            if show_fig:
              fig.show(config={"scrollZoom": True})
        except Exception as e:
            pass

    print(f"Total: Win:{GLOBAL_STATE["win"]} Loss:{GLOBAL_STATE["loss"]}  PnL:{GLOBAL_STATE["pnl"]}%\n")

In [19]:
def run_multi_range(ticker_symbol, days_back, show_fig=False):
  for i in ticker_symbol:
    run_range(i, days_back, show_fig)

In [20]:
#Main entry

# For testing single
ticker_symbol = "HDFCBANK.NS"
date = "2025-12-19"
data_1h = get_single_symbol_day("HDFCBANK.NS","2025-12-19","1h")
data_5m = get_single_symbol_day("HDFCBANK.NS","2025-12-19","5m")
fig = strategy1(ticker_symbol, date, data_1h, data_5m)
fig.show(config={"scrollZoom": True})

#Old executions

#run_multi_range(get_ticker_list(),60, False)

# For testing single over range
#ticker_symbol = "TITAN.NS"
#run_range(ticker_symbol,60,True)

# For testing single
#ticker_symbol = "HDFCBANK.NS"
#date = "2025-12-19"
#fig = strategy1(ticker_symbol, date);
#fig.show(config={"scrollZoom": True})

#data = get_data("HDFCBANK.NS","2025-12-19","5m")
#print(type(data))
#display(data)

#data1 = get_single_symbol_day("HDFCBANK.NS","2025-12-19","5m")
#print(type(data1))
#display(data1)

#data2 = get_multiple_symbols_day(["HDFCBANK.NS","TITAN.NS"], "2025-12-29","5m")
#df1 = extract_ticker(data2, "HDFCBANK.NS")
#df1 = data2.xs("HDFCBANK.NS", axis=1, level=1)
#print(type(df1))
#display(df1)